First we map the companies inside the ad data to specific market sectors.

We perform API calls to On-Demand agents to perform a lookup of the company name and return the sector. We then store the results in a dictionary to avoid repeated calls.

The API calls are performed in parallel to speed up the process.

In [ ]:
from buda.analysis import map_companies
from buda.utils import load_data

data =load_data("/Users/cakir/Documents/projects/buda/dummy_data/advertisers_using_your_activity.json")


map_companies(data, api_key="ekaATB5SfNplUO9FiyfCdx5snqEJgZBD", debug=True)


100%|██████████| 210/210 [03:44<00:00,  1.07s/it]


Create statistics for the companies and categories they belong to

In [1]:
from collections import Counter
import pandas as pd
from buda.utils import load_data

# Provided JSON data
data = load_data("company_analysis_output/categorized_data.json")

# Count the occurrences of each category
category_counts = Counter(data.values())

# Convert to DataFrame for easier visualization
df = pd.DataFrame(list(category_counts.items()), columns=["Category", "Count"])

# Display the DataFrame
print(df)

                                             Category  Count
0                                               Other    137
1                                           Education     27
2                             Media and Entertainment    358
3                                          Technology     76
4                                              Travel     11
5                                          Healthcare     27
6                                  Fashion and Beauty    148
7                                              Retail     59
8                                             Unknown    141
9                                         Real Estate      7
10                                         E-commerce     18
11                                  Food and Beverage     47
12                                         Consulting     77
13                                            Finance     27
14                                 Sports and Fitness     22
15                      

Map the categorize to subcategories:

In [ ]:
import json
category_mapping = {
    'Media and Entertainment': 'Media and Entertainment',
    'Fashion and Beauty': 'Media and Entertainment',
    'Gaming': 'Media and Entertainment',
    'Photography': 'Media and Entertainment',
    'Sports and Fitness': 'Media and Entertainment',
    'Consulting': 'Business and Professional Services',
    'Real Estate': 'Business and Professional Services',
    'Advertising': 'Business and Professional Services',
    'Finance': 'Business and Professional Services',
    'Non-profit': 'Business and Professional Services',
    'Technology': 'Technology and E-commerce',
    'E-commerce': 'Technology and E-commerce',
    'Retail': 'Technology and E-commerce',
    'Healthcare': 'Health and Wellness',
    'Food and Beverage': 'Health and Wellness',
    'PetSafe Brand': 'Health and Wellness',
    'Travel': 'Travel, Hospitality, and Transportation',
    'Hospitality': 'Travel, Hospitality, and Transportation',
    'Transportation': 'Travel, Hospitality, and Transportation',
    'Energy': 'Energy and Utilities',
    'Unknown': 'Uncategorized',
    'Other': 'Uncategorized'
}

# Map categories and group by new categories
df['Broad Category'] = df['Category'].map(category_mapping)
summary_df = df.groupby('Broad Category')['Count'].sum().reset_index()

# Convert the result to JSON
result = summary_df.set_index('Broad Category')['Count'].to_dict()
with open('summary_categories.json', 'w') as f:
    json.dump(result, f, indent=4)

print("Summary saved to summary_categories.json")
print(result)

Summary saved to summary_categories.json


All of this is done inside the analysis function.

In [2]:
from buda.analysis.companies import analyze_companies

input_folder = "company_analysis_output_ufuk"  # folder containing categorized_data.json
output_folder = "../data"
filename = "advertisers_using_your_activity"

analyze_companies(input_folder=input_folder, output_folder=output_folder, filename=filename)

{'Business and Professional Services': 17,
 'Energy and Utilities': 1,
 'Health and Wellness': 9,
 'Media and Entertainment': 87,
 'Technology and E-commerce': 21,
 'Travel, Hospitality, and Transportation': 3,
 'Uncategorized': 67}

This will save the analysis in JSON format to the Data folder, which is then rendered in the frontend